In [303]:
# importing necessary packages

import pickle
import os
import numpy as np
import pandas as pd
from sklearn import neural_network, linear_model, cluster

np.random.seed(0) # seeding 0 for random number generation

In [304]:
# importing the adult dataset, it is split into train and test by default
a = pd.read_csv('adult/adult.data', header=None,names=['age', 'workclass', 'fnlwgt', 'education',
                       'education-num', 'marital-status', 'occupation', 'relationship',
                       'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
                       'native-country', 'income']) # dim: (32561 x 15)

b = pd.read_csv('adult/adult.test', header=None,
                names=['age', 'workclass', 'fnlwgt', 'education',
                       'education-num', 'marital-status', 'occupation', 'relationship',
                       'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
                      'native-country', 'income'])# dim: (16282, 15)


In [305]:
# concatenate the data to preprocess the whole dataset according to our need

total = pd.concat([a, b], axis=0) # dim: (488843, 15)

# dropping education, native country, fnlwgt as mentioned in the paper ( three columns basically)
total = total.drop('education', axis=1)
total = total.drop('native-country', axis=1)
total = total.drop('fnlwgt', axis=1)

# dim: (488843, 12)
# print(total['income'])

# adding new columns for columns with categorical variables
for col in ['workclass', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'income']:
    if '-' in col:
        prefix_col = col.split('-')[0]
    else:
        prefix_col = col
    
    total = pd.concat([total, pd.get_dummies(total[col], prefix=prefix_col, drop_first=True)], axis=1)
    total = total.drop(col, axis=1)

# dim: (488843, 60)

print(total.shape)
print(list(total.columns))
total.tail()

(48843, 60)
['age', 'capital-gain', 'capital-loss', 'hours-per-week', 'workclass_ Federal-gov', 'workclass_ Local-gov', 'workclass_ Never-worked', 'workclass_ Private', 'workclass_ Self-emp-inc', 'workclass_ Self-emp-not-inc', 'workclass_ State-gov', 'workclass_ Without-pay', 'education_2.0', 'education_3.0', 'education_4.0', 'education_5.0', 'education_6.0', 'education_7.0', 'education_8.0', 'education_9.0', 'education_10.0', 'education_11.0', 'education_12.0', 'education_13.0', 'education_14.0', 'education_15.0', 'education_16.0', 'marital_ Married-AF-spouse', 'marital_ Married-civ-spouse', 'marital_ Married-spouse-absent', 'marital_ Never-married', 'marital_ Separated', 'marital_ Widowed', 'occupation_ Adm-clerical', 'occupation_ Armed-Forces', 'occupation_ Craft-repair', 'occupation_ Exec-managerial', 'occupation_ Farming-fishing', 'occupation_ Handlers-cleaners', 'occupation_ Machine-op-inspct', 'occupation_ Other-service', 'occupation_ Priv-house-serv', 'occupation_ Prof-specialt

,age,capital-gain,capital-loss,hours-per-week,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,...,relationship_ Unmarried,relationship_ Wife,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White,sex_ Male,income_ <=50K.,income_ >50K,income_ >50K.
16277,39,0.0,0.0,36.0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,1,0,0
16278,64,0.0,0.0,40.0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,1,0,0
16279,38,0.0,0.0,50.0,0,0,0,1,0,0,...,0,0,0,0,0,1,1,1,0,0
16280,44,5455.0,0.0,40.0,0,0,0,1,0,0,...,0,0,1,0,0,0,1,1,0,0
16281,35,0.0,0.0,60.0,0,0,0,0,1,0,...,0,0,0,0,0,1,1,0,0,1


In [306]:
# converting the pandas dataframe to numpy array
total_np = total.to_numpy()

# D has 7841 samples, D_aux has 7841 samples, and D_test has 7692 samples

# taking '>50K' income as y values
y = (total_np[:, -1] + total_np[:, -2]).astype(np.float32) # last two columns are duplicate, so add them 
y = np.delete(y, 32561, axis=0)                            # delete the 32561th row value for y as it has NaN values

# taking rest of the data as x values, after deleting the last three columns i.e columns
x = np.delete(total_np, [total_np.shape[1]-1, total_np.shape[1]-2, total_np.shape[1]-3], axis=1)
x = np.delete(x, 32561, axis=0).astype(np.float32)         # delete the 32561th row value for x as it has NaN vlaues


#separating the dataset into training and testing, training [0-32560] row, testing [32561-rest]
train_x, train_y = x[:32561], y[:32561]     # dim: (32561 x 57)
test_x, test_y = x[32561:], y[32561:]       # dim: (16281 x 57)

# saving the index of samples where y is true(1) and false(0)
train_zero_inds = np.where(train_y==0)[0]   # dim: (24720 x 1)
train_one_inds = np.where(train_y==1)[0]    # dim: (7841 x 1)
test_zero_inds = np.where(test_y==0)[0]     # dim: (12435 x 1)
test_one_inds = np.where(test_y==1)[0]      # dim: (3846 x 1)

# creating an array of random numbers, range[0-24720(train_zero_inds.shape[0])], of dimension (7841(train_one_inds.shape[0]), 1) 
train_zeros = np.random.choice(train_zero_inds.shape[0], train_one_inds.shape[0], replace=False) # dim:(7841 x 1)

# creating an array of random numbers, range[0-12435(test_zero_inds.shape[0])], of dimension (3846(test_one_inds.shape[0]), 1)
test_zeros = np.random.choice(test_zero_inds.shape[0], test_one_inds.shape[0], replace=False) # dim: (3846 x 1)


# concatenating random choices of zero indexed example with one indexed example to build a dataset with 
# equal number of zero and one indexed samples
train_x = np.concatenate((train_x[train_zeros], train_x[train_one_inds]), axis=0) # dim: (15682 x 57)
train_y = np.concatenate((train_y[train_zeros], train_y[train_one_inds]), axis=0) # dim: (15682 x 1)


test_x = np.concatenate((test_x[test_zeros], test_x[test_one_inds]), axis=0)      # dim: (7692 x 57)
test_y = np.concatenate((test_y[test_zeros], test_y[test_one_inds]), axis=0)      # dim: (7692 x 1)


# shuffle training data(row wise) by shuffling the index and getting the data from shuffled index
train_shuffle = np.random.choice(train_x.shape[0], train_x.shape[0], replace=False) # dim: (15682 x 1)
train_x, train_y = train_x[train_shuffle], train_y[train_shuffle]                   # dim: (15682 x 57) and (15682 x 1)

train_size = train_x.shape[0]//2

train_x, train_y, ho_x, ho_y = train_x[:train_size], train_y[:train_size], train_x[train_size:], train_y[train_size:]

pois_rates = [0.5, 1, 2]     # poison rates
#pois_rates = [0.5, 1, 1.5, 2, 2.5, 3]

# D: (train_x, train_y)
# D_aux : (ho_x, ho_y)
# D_test: (test_x, test_y)
train_x.shape, train_y.shape, ho_x.shape, ho_x.shape, test_x.shape, test_y.shape

((7841, 57), (7841,), (7841, 57), (7841, 57), (7692, 57), (7692,))

In [307]:
# training and testing the linear regression model
lm = linear_model.LogisticRegression(max_iter = 5000)
lm.fit(train_x, train_y)
lm.score(test_x, test_y)

0.8364534581383255

In [308]:
# saving the trained model as a pickel file
save_path = 'subpopulation/'

with open(os.path.join(save_path,"lm.pickle"), "wb") as f:
    pickle.dump(lm,f)

# # loading the trained model 
# with open(os.path.join(save_path,"lm.pickle"), "rb") as f:
#     test = pickle.load(f)

In [309]:
# print(lm.classes_) # two classes '0' & '1'

# predict the output probability for ho_x dataset. predict_proba gives probability for each class i.e a tuple 
# in this case [0.1 0.9] probaibility for this class. np.eye(2) gets [1 0] when y = 0 and [0 1] when y = 1
# which when muliplied singles out the probability
lm_preds = np.multiply(lm.predict_proba(ho_x), np.eye(2)[ho_y.astype(np.int)]).sum(axis=1) # dim: (7841, 1)

# could have simply done predict
lm_pred_class = lm.predict(ho_x)                    # dim: (7841, 1)


print(lm_preds) # print the probabilites but has no information on which class it belongs to
print(lm_pred_class)  # print the predictions, gives classes


[0.97870038 0.97758587 0.05441383 ... 0.94849411 0.95495388 0.67978749]
[0. 1. 0. ... 0. 1. 0.]


In [310]:
# training and test the neural network with 10 hidden units
nn = neural_network.MLPClassifier(hidden_layer_sizes=(10,))
nn.fit(train_x, train_y)
clean_test_acc = nn.score(test_x, test_y)
print(clean_test_acc)

0.8337233489339574


In [311]:
# prediction for the nn

# probability of belonging to a clas

nn_preds = np.multiply(nn.predict_proba(ho_x), np.eye(2)[ho_y.astype(np.int)]).sum(axis=1) 
nn_pred_class = nn.predict(ho_x) # prediction class

print(nn_preds)
print(nn_pred_class)

[0.97376835 0.9968766  0.08229731 ... 0.96648937 0.99939501 0.75527519]
[0. 1. 0. ... 0. 1. 0.]


In [312]:
pred_mean = np.multiply(nn.predict_proba(test_x), np.eye(2)[test_y.astype(int)]).mean()*2
print(pred_mean)

0.7632116561694878


In [317]:
# FeatureMatch Data Preprocessing

np.random.seed(0)
# concatenating education level and race for D_aux data
protected = np.concatenate((ho_x[:, 12:27], ho_x[:, 52:57]), axis=1) # dim: (7841, 20)

# concatenating education level and race for D_test data
test_prot = np.concatenate((test_x[:, 12:27], test_x[:, 52:57]), axis=1) # dim: (7692, 20)

# concatenating education level and race for D_train data
train_prot = np.concatenate((train_x[:, 12:27], train_x[:, 52:57]), axis=1) # dim: (7841, 20)

all_cols = list(total.columns)                # getting names of all columns
prot_cols = all_cols[12:27] + all_cols[52:57] # getting names of protected columns

subclasses, counts = np.unique(protected, axis=0, return_counts=True)# 122 unique examples in the protected data 
# dim: (122 x 20), (122,1)
# print(tuple(zip(subclasses, counts)))
# print(subclasses[0:5], counts[0:5])

hd_sbcl_conf = []                             # empty array
hd_used = []                                  # empty array

# decalring arrays for storing errors in the subclasses, i.e [subclasses, (clean_acc, collat, target), pois_ind]

#  dim: (122 x 3 x len: pois rates) for a class (3 x pois len) dimensional array
hd_lr_errs = np.zeros((len(subclasses), 3, len(pois_rates)))  
hd_nn_errs = np.zeros((len(subclasses), 3, len(pois_rates)))  #  dim: (122 x 3 x len: pois rates) for a class (3 x 3 pois len) dimensional array


In [314]:
# for subclasses[1] to understand the algorithm 

print(subclasses[1], counts[1])

test_sbcl = np.where(np.linalg.norm(test_prot - subclasses[1], axis=1)==0)
# print(test_sbcl)

sbcl = np.where(np.linalg.norm(protected - subclasses[1], axis=1)==0)
# print(sbcl)

train_sbcl = np.where(np.linalg.norm(train_prot - subclasses[1], axis=1)==0)
# print(train_sbcl)

p_t_x, p_t_y = test_x[test_sbcl], test_y[test_sbcl]

pois_x_base, pois_y_base = ho_x[sbcl], ho_y[sbcl]
# print(p_t_x, p_t_y)
# print(p_t_x[0].shape)
# print(p_t_x[0][12:27])
# print(p_t_x[1][12:27])
# print(p_t_x[2][12:27])
# print(pois_x_base, pois_x_base)

sc_lr_pred, sc_nn_pred = lm_preds[sbcl].mean(), nn_preds[sbcl].mean()
# print(sc_lr_pred, sc_nn_pred)

train_ct = train_sbcl[0].shape[0]
test_ct = p_t_x.shape[0]
print("Identical training samples: ", train_ct)
print("Identical aux samples: ", pois_x_base.shape[0])
print("Identical testing samples: ", test_ct)
print("\n")

t = 2*np.multiply(lm.predict_proba(p_t_x), np.eye(2)[p_t_y.astype(int)]).mean()
# print(t)

# for v, prot_col in zip(subclasses[1], prot_cols):
#     if v> 0.5:
#         print(v, prot_col)

pois_ct = [int(train_ct*pois_rate) for pois_rate in pois_rates]
print(pois_ct)
for j, pois_ct in enumerate([int(train_ct*pois_rate) for pois_rate in pois_rates]):
    
    pois_inds = np.random.choice(pois_x_base.shape[0], pois_ct, replace=True)
    pois_x, pois_y = pois_x_base[pois_inds], 1-pois_y_base[pois_inds]
    total_x, total_y = np.concatenate((train_x, pois_x), axis=0), np.concatenate((train_y, pois_y), axis=0)
    
    print([prot_col for v, prot_col in zip(subclasses[1], prot_cols) if v > 0.5], train_ct, pois_ct, test_ct)
    print("poison fraction:", pois_ct/train_ct) # poison fraction
    
    lmps = [linear_model.LogisticRegression(solver='liblinear', max_iter=500) for _ in range(3)] 
    for lmp in lmps:
            lmp.fit(total_x, total_y)
    
    nnps = [neural_network.MLPClassifier(hidden_layer_sizes=(10,), max_iter=3000) for _ in range(3)]
    for nnp in nnps:
        nnp.fit(total_x, total_y)
        
    lmp_acc = np.mean([lmp.score(test_x, test_y) for lmp in lmps])
    print("lr acc {:.3f}".format(lmp_acc))
    
    nnp_acc = np.mean([nnp.score(test_x, test_y) for nnp in nnps])
    print("nn acc {:.3f}".format(nnp_acc))
    
    # checking the score for 
    lmc_sbc = lm.score(p_t_x, p_t_y)
    print("lr cl sbc {:.3f}".format(lmc_sbc))
        
    lmp_sbc = np.mean([lmp.score(p_t_x, p_t_y) for lmp in lmps])
    print("lr sbc {:.3f}".format(lmp_sbc))

    nnc_sbc = nn.score(p_t_x, p_t_y)
    print("nn cl sbc {:.3f}".format(nnc_sbc))

    nnp_sbc = np.mean([nnp.score(p_t_x, p_t_y) for nnp in nnps])
    print("nn sbc {:.3f}".format(nnp_sbc))
#     hd_lr_errs[i, 0, j] = lmc_sbc
#     hd_lr_errs[i, 1, j] = lmp_acc
#     hd_lr_errs[i, 2, j] = lmp_sbc

#     hd_nn_errs[i, 0, j] = nnc_sbc
#     hd_nn_errs[i, 1, j] = nnp_acc
#     hd_nn_errs[i, 2, j] = nnp_sbc
    
    

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.] 5
Identical training samples:  3
Identical aux samples:  5
Identical testing samples:  3


[1, 3, 6]
['race_ White', 'sex_ Male'] 3 1 3
poison fraction: 0.3333333333333333
lr acc 0.837
nn acc 0.830
lr cl sbc 1.000
lr sbc 0.667
nn cl sbc 0.667
nn sbc 0.667
['race_ White', 'sex_ Male'] 3 3 3
poison fraction: 1.0
lr acc 0.834
nn acc 0.833
lr cl sbc 1.000
lr sbc 0.667
nn cl sbc 0.667
nn sbc 0.667
['race_ White', 'sex_ Male'] 3 6 3
poison fraction: 2.0
lr acc 0.833
nn acc 0.829
lr cl sbc 1.000
lr sbc 0.667
nn cl sbc 0.667
nn sbc 0.667


In [315]:
# Feature Match: Initially there are 122 subclasses of common features, later if senctences filters the subclasses 
# to number to 35

for i, (subcl, count) in enumerate(zip(subclasses, counts)):  # for each subclass in subclasses
    if count > 10 and count < 100:                            # if the number of counts is more than 10 and less than 100
        hd_used.append((i, count))                            # mark if by apppend to hd_used
        
        # subtract the current subclass from test_prot data, find frobenius norm along columns and then get index of
        # data where the norm is still zero. This finds the samples in test_prot (D_test) which have identical features 
        # to current subclass
        test_sbcl = np.where(np.linalg.norm(test_prot - subcl, axis=1)==0) # 2D array of (indexes, datatypes)
        
        # same logic as above, finding the samples in protected data (D_aux) which have identical features to current subclass
        sbcl = np.where(np.linalg.norm(protected-subcl, axis=1)==0)   # 2D array of (indexes, datatypes) Note: number of index
                                                                      # should be equal to counts value
        
        # same logic as above, finding the samples in protected data (D_train) which have identical features to current subclass
        train_sbcl = np.where(np.linalg.norm(train_prot - subcl, axis=1)==0) # 2D array of (indexes, datatypes)
        
        # getting the samples with idential feature (to current subclass) from test data (D_test)
        p_t_x, p_t_y = test_x[test_sbcl], test_y[test_sbcl] 
                                                            
        # getting the samples with idential feature (to current subclass) from auxiliary data (D_aux)
        # labelling it as poison data
        pois_x_base, pois_y_base = ho_x[sbcl], ho_y[sbcl]  
         
        # getting the prediction probability of identical samples from(D_aux), and finding their mean
        sc_lr_pred, sc_nn_pred = lm_preds[sbcl].mean(), nn_preds[sbcl].mean()
        print(sc_lr_pred, sc_nn_pred)
        
        train_ct = train_sbcl[0].shape[0] # number of identical samples in train_sbcl (D_train)
        test_ct = p_t_x.shape[0]          # number of identical samples in test_sbcl (D_test) 
        
        # multiplying prediction probability [x y] of a sample with [0 1] or [1 0] based on y values
        # taking their sum(mean) and multiplying with 2
        hd_sbcl_conf.append(2*np.multiply(lm.predict_proba(p_t_x), np.eye(2)[p_t_y.astype(int)]).mean())
    
    all_errs = []
    
    # multiply poison rates with number of identical training samples, then take its integer, dim same as pois rate, 1D array
    # pois_ct now has the number of poisoned samples which needs to be added to D_train
    # for each number(pois_ct) in the pois_rates 
    for j, pois_ct in enumerate([int(train_ct*pois_rate) for pois_rate in pois_rates]):
        
        # get the random indexes of 'pois_ct' number of samples from pois_x_base with replacement (repeated data is allowed)
        pois_inds = np.random.choice(pois_x_base.shape[0], pois_ct, replace=True)
        
        # get the x_values and flip the y values. Now we have the poison data
        pois_x, pois_y = pois_x_base[pois_inds], 1 - pois_y_base[pois_inds]
        
        # add the poisoned data to the training data
        total_x, total_y = np.concatenate((train_x, pois_x), axis=0), np.concatenate((train_y, pois_y), axis=0)
        
        # print the column name for which the value (v > 0.5) is 1 basically, then print the number of samples
        # in training, poisoned, and testing data. This finally prints which features we are poisoning
        print([prot_col for v, prot_col in zip(subcl, prot_cols) if v > 0.5], train_ct, pois_ct, test_ct)
        print("poison fraction:", pois_ct/train_ct) # printing the poison fraction
        
        # creating three Logistic regression model so that we could train three models and average their results
        lmps = [linear_model.LogisticRegression(solver='liblinear', max_iter=5000) for _ in range(3)] 
        for lmp in lmps:
            lmp.fit(total_x, total_y)
        
        # creating three Logistic regression model so that we could train three models on the Poisoned Data (total_x, total_y)
        nnps = [neural_network.MLPClassifier(hidden_layer_sizes=(10,), max_iter=3000) for _ in range(3)]
        for nnp in nnps:
            nnp.fit(total_x, total_y)
        
        # get the average accuracy score on test data (D_test) for three models of LR
        lmp_acc_c = np.mean([lmp.score(test_x, test_y) for lmp in lmps])
        print("Poisoned lr test acc {:.3f}".format(lmp_acc))
        
        # get the average accuracy score on test data (D_test) for three models of NN
        nnp_acc_c = np.mean([nnp.score(test_x, test_y) for nnp in nnps])
        print("Poisoned nn test acc {:.3f}".format(nnp_acc))
        
        # checking the score for identical samples(p_t_x, p_t_y) in test data on Clean LR model
        lmc_sbc = lm.score(p_t_x, p_t_y)
        print("lr clean sbc {:.3f}".format(lmc_sbc))
        
        # checking the score for identical samples(p_t_x, p_t_y) in test data on Poisoned LR model
        lmp_sbc_t = np.mean([lmp.score(p_t_x, p_t_y) for lmp in lmps])
        print("lr poisoned sbc {:.3f}".format(lmp_sbc))
        
        # checking the score for identical samples(p_t_x, p_t_y) in test data on Clean NN model
        nnc_sbc = nn.score(p_t_x, p_t_y)
        print("nn clean  sbc {:.3f}".format(nnc_sbc))
        
        # checking the score for identical samples(p_t_x, p_t_y) in test data on Clean NN model
        nnp_sbc_t = np.mean([nnp.score(p_t_x, p_t_y) for nnp in nnps])
        print("nn poisoned sbc {:.3f}".format(nnp_sbc))
        
        # storing the errors as (subclass, (clean_acc, collat, target), (pois_ind))
        hd_lr_errs[i, 0, j] = lmc_sbc
        hd_lr_errs[i, 1, j] = lmp_acc_c
        hd_lr_errs[i, 2, j] = lmp_sbc_t
        
        hd_nn_errs[i, 0, j] = nnc_sbc
        hd_nn_errs[i, 1, j] = nnp_acc_c
        hd_nn_errs[i, 2, j] = nnp_sbc_t

['race_ White'] 3 1 3
poison fraction: 0.3333333333333333
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 1.000
lr poisoned sbc 0.667
nn clean  sbc 0.667
nn poisoned sbc 0.667
['race_ White'] 3 3 3
poison fraction: 1.0
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 1.000
lr poisoned sbc 0.667
nn clean  sbc 0.667
nn poisoned sbc 0.667
['race_ White'] 3 6 3
poison fraction: 2.0
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 1.000
lr poisoned sbc 0.667
nn clean  sbc 0.667
nn poisoned sbc 0.667
['race_ White', 'sex_ Male'] 3 1 3
poison fraction: 0.3333333333333333
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 1.000
lr poisoned sbc 0.667
nn clean  sbc 0.667
nn poisoned sbc 0.667
['race_ White', 'sex_ Male'] 3 3 3
poison fraction: 1.0
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 1.000
lr poisoned sbc 0.667
nn clean  sbc 0.667
nn poisoned sbc 0.667
['race_ White', 'sex_ Male'] 3 6 3
poison fr

Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 1.000
lr poisoned sbc 0.667
nn clean  sbc 0.955
nn poisoned sbc 0.667
['education_15.0', 'race_ White'] 22 44 22
poison fraction: 2.0
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 1.000
lr poisoned sbc 0.667
nn clean  sbc 0.955
nn poisoned sbc 0.667
['education_15.0', 'race_ White', 'sex_ Male'] 22 11 22
poison fraction: 0.5
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 1.000
lr poisoned sbc 0.667
nn clean  sbc 0.955
nn poisoned sbc 0.667
['education_15.0', 'race_ White', 'sex_ Male'] 22 22 22
poison fraction: 1.0
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 1.000
lr poisoned sbc 0.667
nn clean  sbc 0.955
nn poisoned sbc 0.667
['education_15.0', 'race_ White', 'sex_ Male'] 22 44 22
poison fraction: 2.0
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 1.000
lr poisoned sbc 0.667
nn clean  sbc 0.955
nn poisoned sbc 0.667
['education_15.0', '

Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.947
lr poisoned sbc 0.667
nn clean  sbc 0.947
nn poisoned sbc 0.667
['education_14.0', 'race_ Asian-Pac-Islander'] 16 32 19
poison fraction: 2.0
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.947
lr poisoned sbc 0.667
nn clean  sbc 0.947
nn poisoned sbc 0.667
0.8269123075783553 0.825429288779988
['education_14.0', 'race_ Asian-Pac-Islander', 'sex_ Male'] 35 17 29
poison fraction: 0.4857142857142857
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.759
lr poisoned sbc 0.667
nn clean  sbc 0.793
nn poisoned sbc 0.667
['education_14.0', 'race_ Asian-Pac-Islander', 'sex_ Male'] 35 35 29
poison fraction: 1.0
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.759
lr poisoned sbc 0.667
nn clean  sbc 0.793
nn poisoned sbc 0.667
['education_14.0', 'race_ Asian-Pac-Islander', 'sex_ Male'] 35 70 29
poison fraction: 2.0
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829

Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.738
lr poisoned sbc 0.667
nn clean  sbc 0.738
nn poisoned sbc 0.667
['education_12.0', 'race_ White', 'sex_ Male'] 66 66 65
poison fraction: 1.0
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.738
lr poisoned sbc 0.667
nn clean  sbc 0.738
nn poisoned sbc 0.667
['education_12.0', 'race_ White', 'sex_ Male'] 66 132 65
poison fraction: 2.0
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.738
lr poisoned sbc 0.667
nn clean  sbc 0.738
nn poisoned sbc 0.667
['education_12.0', 'race_ Other'] 66 33 65
poison fraction: 0.5
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.738
lr poisoned sbc 0.667
nn clean  sbc 0.738
nn poisoned sbc 0.667
['education_12.0', 'race_ Other'] 66 66 65
poison fraction: 1.0
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.738
lr poisoned sbc 0.667
nn clean  sbc 0.738
nn poisoned sbc 0.667
['education_12.0', 'race_ Other'

Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.778
lr poisoned sbc 0.667
nn clean  sbc 0.778
nn poisoned sbc 0.667
['education_11.0', 'race_ Black', 'sex_ Male'] 11 11 9
poison fraction: 1.0
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.778
lr poisoned sbc 0.667
nn clean  sbc 0.778
nn poisoned sbc 0.667
['education_11.0', 'race_ Black', 'sex_ Male'] 11 22 9
poison fraction: 2.0
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.778
lr poisoned sbc 0.667
nn clean  sbc 0.778
nn poisoned sbc 0.667
['education_11.0', 'race_ Asian-Pac-Islander'] 11 5 9
poison fraction: 0.45454545454545453
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.778
lr poisoned sbc 0.667
nn clean  sbc 0.778
nn poisoned sbc 0.667
['education_11.0', 'race_ Asian-Pac-Islander'] 11 11 9
poison fraction: 1.0
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.778
lr poisoned sbc 0.667
nn clean  sbc 0.778
nn poisoned sbc 0.

Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.963
lr poisoned sbc 0.667
nn clean  sbc 0.926
nn poisoned sbc 0.667
['education_9.0'] 35 17 27
poison fraction: 0.4857142857142857
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.963
lr poisoned sbc 0.667
nn clean  sbc 0.926
nn poisoned sbc 0.667
['education_9.0'] 35 35 27
poison fraction: 1.0
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.963
lr poisoned sbc 0.667
nn clean  sbc 0.926
nn poisoned sbc 0.667
['education_9.0'] 35 70 27
poison fraction: 2.0
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.963
lr poisoned sbc 0.667
nn clean  sbc 0.926
nn poisoned sbc 0.667
0.7715436078861 0.7231216095387936
['education_9.0', 'sex_ Male'] 13 6 11
poison fraction: 0.46153846153846156
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.818
lr poisoned sbc 0.667
nn clean  sbc 0.818
nn poisoned sbc 0.667
['education_9.0', 'sex_ Male'] 13 13 11
poiso

Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.718
lr poisoned sbc 0.667
nn clean  sbc 0.744
nn poisoned sbc 0.667
['education_8.0', 'race_ Black'] 36 18 39
poison fraction: 0.5
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.718
lr poisoned sbc 0.667
nn clean  sbc 0.744
nn poisoned sbc 0.667
['education_8.0', 'race_ Black'] 36 36 39
poison fraction: 1.0
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.718
lr poisoned sbc 0.667
nn clean  sbc 0.744
nn poisoned sbc 0.667
['education_8.0', 'race_ Black'] 36 72 39
poison fraction: 2.0
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.718
lr poisoned sbc 0.667
nn clean  sbc 0.744
nn poisoned sbc 0.667
['education_8.0', 'race_ Black', 'sex_ Male'] 36 18 39
poison fraction: 0.5
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.718
lr poisoned sbc 0.667
nn clean  sbc 0.744
nn poisoned sbc 0.667
['education_8.0', 'race_ Black', 'sex_ Male'] 36 3

Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.939
lr poisoned sbc 0.667
nn clean  sbc 0.939
nn poisoned sbc 0.667
0.6885488117423945 0.6945606582271664
['education_6.0', 'race_ White', 'sex_ Male'] 98 49 97
poison fraction: 0.5
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.773
lr poisoned sbc 0.667
nn clean  sbc 0.763
nn poisoned sbc 0.667
['education_6.0', 'race_ White', 'sex_ Male'] 98 98 97
poison fraction: 1.0
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.773
lr poisoned sbc 0.667
nn clean  sbc 0.763
nn poisoned sbc 0.667
['education_6.0', 'race_ White', 'sex_ Male'] 98 196 97
poison fraction: 2.0
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.773
lr poisoned sbc 0.667
nn clean  sbc 0.763
nn poisoned sbc 0.667
['education_6.0', 'race_ Other'] 98 49 97
poison fraction: 0.5
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.773
lr poisoned sbc 0.667
nn clean  sbc 0.763
nn pois

Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.787
lr poisoned sbc 0.667
nn clean  sbc 0.787
nn poisoned sbc 0.667
['education_4.0', 'sex_ Male'] 54 27 47
poison fraction: 0.5
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.787
lr poisoned sbc 0.667
nn clean  sbc 0.787
nn poisoned sbc 0.667
['education_4.0', 'sex_ Male'] 54 54 47
poison fraction: 1.0
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.787
lr poisoned sbc 0.667
nn clean  sbc 0.787
nn poisoned sbc 0.667
['education_4.0', 'sex_ Male'] 54 108 47
poison fraction: 2.0
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.787
lr poisoned sbc 0.667
nn clean  sbc 0.787
nn poisoned sbc 0.667
0.8229906793040872 0.8252111971378326
['education_4.0', 'race_ White'] 17 8 11
poison fraction: 0.47058823529411764
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 1.000
lr poisoned sbc 0.667
nn clean  sbc 1.000
nn poisoned sbc 0.667
['education_4.0

Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.771
lr poisoned sbc 0.667
nn clean  sbc 0.771
nn poisoned sbc 0.667
['education_2.0', 'race_ White'] 25 12 35
poison fraction: 0.48
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.771
lr poisoned sbc 0.667
nn clean  sbc 0.771
nn poisoned sbc 0.667
['education_2.0', 'race_ White'] 25 25 35
poison fraction: 1.0
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.771
lr poisoned sbc 0.667
nn clean  sbc 0.771
nn poisoned sbc 0.667
['education_2.0', 'race_ White'] 25 50 35
poison fraction: 2.0
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.771
lr poisoned sbc 0.667
nn clean  sbc 0.771
nn poisoned sbc 0.667
0.7122070447784948 0.757268476817343
['education_2.0', 'race_ White', 'sex_ Male'] 11 5 15
poison fraction: 0.45454545454545453
Poisoned lr test acc 0.833
Poisoned nn test acc 0.829
lr clean sbc 0.667
lr poisoned sbc 0.667
nn clean  sbc 0.800
nn poisoned sbc 0.6

In [ ]:
#Cluster match Data preprocessing

np.random.seed(0)

from sklearn import cluster
km = cluster.KMeans(n_clusters=100)
km.fit(ho_x)
tst_km = km.predict(tst_x)
trn_km = km.predict(trn_x)

kd_sbcl_conf = []
kd_used = []
kd_lr_errs = np.zeros((len(subclasses), 3, len(pois_rates)))  # subcl, (clean_acc, collat, target), pois_ind
kd_nn_errs = np.zeros((len(subclasses), 3, len(pois_rates)))  # subcl, (clean_acc, collat, target), pois_ind

kmeans_designed = []
print(np.unique(km.labels_, return_counts=True))
cl_inds, cl_cts = np.unique(km.labels_, return_counts=True)
print(cl_inds, cl_cts)

In [ ]:


for i, (cl_ind, cl_ct) in enumerate(zip(cl_inds, cl_cts)):
  if cl_ct > 10 and cl_ct < 100:
    kd_used.append((i, cl_ct))

    print(cl_ind, cl_ct, np.where(tst_km==cl_ind)[0].shape[0])
    tst_sbcl = np.where(tst_km==cl_ind)
    sbcl = np.where(km.labels_==cl_ind)
    trn_sbcl = np.where(trn_km==cl_ind)
    p_t_x, p_t_y = tst_x[tst_sbcl], tst_y[tst_sbcl]

    kd_sbcl_conf.append(2*np.multiply(lm.predict_proba(p_t_x),
                                      np.eye(2)[p_t_y.astype(int)]).mean())

        
    pois_x_base, pois_y_base = ho_x[sbcl], ho_y[sbcl]
    
    sc_lr_pred, sc_nn_pred = lm_preds[sbcl].mean(), nn_preds[sbcl].mean()
    
    trn_ct = trn_sbcl[0].shape[0]
    tst_ct = p_t_x.shape[0]
    print(sc_lr_pred, sc_nn_pred)
    all_errs = []

    for j, pois_ct in enumerate([int(trn_ct*pois_rate) for pois_rate in pois_rates]):
      pois_inds = np.random.choice(pois_x_base.shape[0], pois_ct, replace=True)
      pois_x, pois_y = pois_x_base[pois_inds], 1-pois_y_base[pois_inds]

      full_x, full_y = np.concatenate((trn_x, pois_x), axis=0), np.concatenate((trn_y, pois_y), axis=0)
      print("poison fraction:", pois_ct/trn_ct, trn_ct, pois_ct, tst_ct)
      
      lmps = [linear_model.LogisticRegression(solver='liblinear', max_iter=500) for _ in range(3)]
      for lmp in lmps:
        lmp.fit(full_x, full_y)
      nnps = [neural_network.MLPClassifier(hidden_layer_sizes=(10,), max_iter=3000) for _ in range(3)]
      for nnp in nnps:
        nnp.fit(full_x, full_y)

      lmp_acc = np.mean([lmp.score(tst_x, tst_y) for lmp in lmps])
      print("lr acc {:.3f}".format(lmp_acc))

      nnp_acc = np.mean([nnp.score(tst_x, tst_y) for nnp in nnps])
      print("nn acc {:.3f}".format(nnp_acc))

      lmc_sbc = lm.score(p_t_x, p_t_y)
      print("lr cl sbc {:.3f}".format(lmc_sbc))

      lmp_sbc = np.mean([lmp.score(p_t_x, p_t_y) for lmp in lmps])
      print("lr sbc {:.3f}".format(lmp_sbc))

      nnc_sbc = nn.score(p_t_x, p_t_y)
      print("nn cl sbc {:.3f}".format(nnc_sbc))

      nnp_sbc = np.mean([nnp.score(p_t_x, p_t_y) for nnp in nnps])
      print("nn sbc {:.3f}".format(nnp_sbc))
      kd_lr_errs[i, 0, j] = lmc_sbc
      kd_lr_errs[i, 1, j] = lmp_acc
      kd_lr_errs[i, 2, j] = lmp_sbc

      kd_nn_errs[i, 0, j] = nnc_sbc
      kd_nn_errs[i, 1, j] = nnp_acc
      kd_nn_errs[i, 2, j] = nnp_sbc